In [2]:
from app.dotenv import base_dir, data_dir
video_fname = 'vtest.avi'
video_add = f"{data_dir}/{video_fname}"

In [3]:
# from app.dotenv import base_dir, data_dir
# video_fname = 'vtest.avi'
# video_add = f"{data_dir}/{video_fname}"

# import numpy as np 
# import cv2 
  
# cap = cv2.VideoCapture(video_add) 
  
# # initializing subtractor  
# fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()  
  
# while(1): 
#     ret, frame = cap.read()        
  
#     # applying on each frame 
#     fgmask = fgbg.apply(frame) 
      
  
#     cv2.imshow('frame', fgmask) 
#     k = cv2.waitKey(30) & 0xff
#     if k == 27: 
#         break
  
# cap.release() 
# cv2.destroyAllWindows() 

In [4]:
import os
import cv2
import numpy as np

# Assuming you have the folder paths for background learning and testing
folder1_add = f'{data_dir}\{video_fname.split(".")[0]}'  # Folder with images for background modeling
folder2_add = f'{data_dir}\{video_fname.split(".")[0]}'  # Folder with images for foreground subtraction

# List all image files in both folders
image_files1 = sorted([f for f in os.listdir(folder1_add) if f.endswith(('.jpg', '.png'))])
image_files2 = sorted([f for f in os.listdir(folder2_add) if f.endswith(('.jpg', '.png'))])

# Initialize the background subtractor
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

# 1. Calculate the background model from images in folder1_add
for image_file in image_files1:
    image_path = os.path.join(folder1_add, image_file)
    
    # Read the image from folder1_add
    frame = cv2.imread(image_path)
    
    # Check if the frame was successfully loaded
    if frame is None:
        print(f"Error loading image: {image_path}")
        continue
    
    # Apply the background subtractor to learn the background
    fgbg.apply(frame)

print("Background model calculation complete.")




Background model calculation complete.


In [4]:
# 2. After processing the frames, approximate the background model:
# Since OpenCV's MOG doesn't provide direct access to the background model,
# we simulate it by applying the subtractor and using the "background" state.

# Process the last frame or a selected frame after learning:
# fname = image_files1[-1]
backgrounds = []
for fname in image_files1[:10]:
    frame = cv2.imread(os.path.join(folder1_add, fname))  # Get the last frame
    cv2.imshow('frame', frame)
    cv2.waitKey(0)

    # Apply the background subtractor to this frame to get the background state
    fgmask = fgbg.apply(frame)

    # Use the fgmask as a "background" image
    backgrounds.append(cv2.bitwise_and(frame, frame, mask=255-fgmask))



for background in backgrounds:
    cv2.imshow('Background Model', background)
    cv2.waitKey(0)




# # Alternatively, if you want to directly use the fgmask as a simple background:
# # background = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)

# # Save the background image
# save_path = os.path.join(folder1_add, 'background_model.png')  # Or .jpg if preferred
# cv2.imwrite(save_path, background)
# print(f"Background model saved as {save_path}")

[array([[[177, 142, 104],
         [178, 143, 105],
         [178, 143, 105],
         ...,
         [107,  83,  57],
         [106,  80,  70],
         [ 77,  51,  41]],
 
        [[178, 143, 105],
         [178, 143, 105],
         [178, 143, 105],
         ...,
         [105,  81,  55],
         [107,  81,  71],
         [ 80,  54,  44]],
 
        [[178, 143, 105],
         [178, 143, 105],
         [178, 143, 105],
         ...,
         [102,  76,  51],
         [107,  84,  73],
         [ 83,  60,  49]],
 
        ...,
 
        [[ 20,  35,   0],
         [ 20,  35,   0],
         [ 21,  36,   0],
         ...,
         [ 60,  72,   9],
         [ 62,  74,  11],
         [ 62,  74,  11]],
 
        [[ 20,  35,   0],
         [ 20,  35,   0],
         [ 21,  36,   0],
         ...,
         [ 59,  71,   8],
         [ 60,  72,   9],
         [ 62,  74,  11]],
 
        [[ 20,  35,   0],
         [ 20,  35,   0],
         [ 21,  36,   0],
         ...,
         [ 60,  72,   9],
  

In [5]:
# 2. Apply the background model to images in folder2_add and display the foreground subtraction
for image_file in image_files2:
    image_path = os.path.join(folder2_add, image_file)
    
    # Read the image from folder2_add
    frame = cv2.imread(image_path)
    
    # Check if the frame was successfully loaded
    if frame is None:
        print(f"Error loading image: {image_path}")
        continue
    
    # Apply the background subtractor to detect foreground (subtract background)
    fgmask = fgbg.apply(frame)
    
    # Resize the mask to the same size as the original frame for display purposes (optional)
    fgmask_resized = cv2.resize(fgmask, (frame.shape[1], frame.shape[0]))
    
    # Add the filename text to the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, f'Filename: {image_file}', (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # Concatenate the frame, foreground mask, and filename horizontally (side by side)
    side_by_side = np.hstack((frame, cv2.cvtColor(fgmask_resized, cv2.COLOR_GRAY2BGR)))  # Convert fgmask to BGR for display
    
    # Display the concatenated image
    cv2.imshow('Frame and Foreground Mask', side_by_side)
    
    # Wait for 30 ms, and exit if the 'Esc' key is pressed
    k = cv2.waitKey(30) & 0xFF
    if k == 27:  # Esc key to exit
        break

# Clean up
cv2.destroyAllWindows()

: 